In [205]:
with open("/Users/paigegiese/Downloads/html_file_1_s3629110000(1).html", "r", encoding="utf-8") as f:
    landtext = f.read()  
with open("/Users/paigegiese/Downloads/montauk_ex.html", "r", encoding="utf-8") as f:
    restext = f.read()
with open("/Users/paigegiese/Downloads/flying_m_ex.html", "r", encoding="utf-8") as f:
    commtext = f.read()


In [212]:
from bs4 import BeautifulSoup
import re
def extract_full_property_summary(html: str):
    soup = BeautifulSoup(html, "html.parser")
    results = {
        "parcel": None,
        "parcel_status": None,
    }

    # --- 1. Extract Parcel and Parcel Status ---
    for div in soup.find_all("div", class_="col-xxl-2"):
        if "Parcel:" in div.text:
            strong = div.find("strong")
            if strong:
                results["parcel"] = strong.get_text(strip=True)
        elif "Parcel Status:" in div.text:
            span = div.find("span")
            if span:
                results["parcel_status"] = span.get_text(strip=True)

    # --- 2. Extract Accordion Sections ---
    for item in soup.select(".accordion-item"):
        header_btn = item.select_one(".accordion-button")
        section_title = header_btn.get_text(strip=True) if header_btn else "Untitled Section"

        body = item.select_one(".accordion-body")
        if not body:
            continue
        section = {
            section_title: {}
        }

        # Inline <b>Label: value</b> pairs
        for b_tag in body.find_all("b"):
            label = b_tag.get_text(strip=True).rstrip(":")
            next_sibling = b_tag.next_sibling

            if next_sibling and isinstance(next_sibling, str):
                value = next_sibling.strip()
                if value and label != 'Note':
                    section[section_title].update({label: value})
                    #section["inline_fields"][label] = value
        #section.update(inline_fields)
        #print(section)

        # Tables inside the section
      
        for table in body.find_all("table"):
            tbl = {}
            headers = [th.get_text(strip=True) for th in table.select("thead th")]
            #print(headers)
            rows = []
            for tr in table.select("tbody tr"):
                cells = [td.get_text(strip=True) for td in tr.select("td")]
                #print(cells)
                if len(cells) == len(headers):
                    rows.append(dict(zip(headers, cells)))
            tbl.update({table.get("id"):rows})
            #print(tbl)
        
            section[section_title].update(tbl)
        #print(section)
            
    
        results.update(section)
        

    return results


In [259]:
prepped = extract_full_property_summary(landtext)
new_characteristics = []
# quick clean up
for k,v in prepped['Characteristics'].items():
    new_format={}
    for trait in v:
        new_format.update({trait['Characteristic']: trait['Value']})
    
    #print(new_format)
    replacement = {k:new_format}
    new_characteristics.append(replacement)

    #print(v)
prepped.pop('Characteristics')
prepped.update({'characteristics': new_characteristics})

def clean_name(strvalue):
    if bool(re.match(r'^[a-z]+[A-Z]', strvalue)):
        s1 = re.sub(r'(.)([A-Z][a-z]+)', r'\1_\2', strvalue)
        
        clean = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    else:
        clean = strvalue.lower().replace(' ','_').replace('/','_')
    return clean

def clean_keys(obj, clean_func):
    if isinstance(obj, dict):
        return {clean_func(k): clean_keys(v, clean_func) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_keys(item, clean_func) for item in obj]
    else:
        return obj
    
final = clean_keys(prepped, clean_name)





In [260]:
import json
with open('/Users/paigegiese/SYG/landproDATA_code/misc-work/parse/samples/land_tax_sample.json', 'w') as fp:
    json.dump(final, fp)

In [261]:
final

{'parcel': 'S3629110000',
 'parcel_status': 'Active',
 'property_details': {'primary_owner': 'USA (BUREAU OF LAND MANAGEMENT)',
  'address': 'S CINDER BUTTE RD    BOISE, ID 837160000',
  'instrument_#': '0',
  'subdivision': '3S 4E 29',
  'assessor_id': 'SEC 29 3S 4E',
  'township_range_section': '3S4E29',
  'land_group_type': 'SECT',
  'zone_code': 'RP',
  'total_acres': '640.000',
  'tax_code_area': '52'},
 'valuation': {'valuation_details': [{'roll': 'Property',
    'state_category_code': '180',
    'acreage': '640.000',
    'assessed_value': '0',
    'valuation_method': 'MARKET',
    'code_area': '52'}]},
 'tax_districts': {'total_levy': '0.005359265',
  'tax_districts': [{'tax_district': '1',
    'levy': '0.001544150',
    'description': 'ADA COUNTY',
    'phone': '2082877000'},
   {'tax_district': '2',
    'levy': '0.000068079',
    'description': 'PEST EXTERMINATION',
    'phone': '2085774646'},
   {'tax_district': '3',
    'levy': '0.000084684',
    'description': 'EMERGENCY ME